## Keras Auto Encoder for Anomaly Detection with Custom Layer for determining Encoder Decoder Reconstruction Cost## Kohler - Multi-input Categorical Embeddings with Numericals

### Loading Necessary Files and Libraries

In [1]:
import keras
from keras.models import Model, Sequential
from keras.layers import Embedding, Input, Dense, LSTM
from keras.preprocessing.text import Tokenizer
from keras import layers
import pandas as pd
import numpy as np
from sklearn import preprocessing
import matplotlib.pyplot as plt
from keras import optimizers
import pickle
import os
from keras.layers import initializers 
from keras.layers import regularizers 
from keras.layers import constraints 
from keras.layers import Activation
from keras.layers.advanced_activations import PReLU
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.advanced_activations import ELU
from keras.constraints import max_norm
from keras.layers import LeakyReLU
from keras import optimizers, losses, activations, models
from keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler
from keras.layers import Dense, Input, Dropout, Convolution1D, MaxPool1D, GlobalMaxPool1D, GlobalAveragePooling1D, \
    concatenate
from keras.layers import MaxPooling2D, Flatten, Conv2D
from keras.utils import to_categorical
initializer1 = keras.initializers.RandomNormal
initializer2 = keras.initializers.Zeros()
initializer3 = keras.initializers.glorot_uniform(seed=None)
initializer4 = keras.initializers.lecun_normal(seed=None)
%matplotlib inline

Using TensorFlow backend.


## Load Data

In [2]:
data = pd.read_csv("cardio.csv")
data.drop("key", inplace=True, axis=1)
print(data.shape)

(1831, 22)


In [3]:
data.head(1)

,X.1,X.2,X.3,X.4,X.5,X.6,X.7,X.8,X.9,X.10,...,X.13,X.14,X.15,X.16,X.17,X.18,X.19,X.20,X.21,target
0,0.004912,0.693191,-0.20364,0.595322,0.35319,-0.061401,-0.278295,-1.650444,0.759072,-0.420487,...,-0.798376,1.854728,0.622631,0.963083,0.301464,0.193113,0.231498,-0.289786,-0.493294,0


In [4]:
data["target"].value_counts()

0    1655
1     176
Name: target, dtype: int64

In [5]:
all = data
positive = data[data["target"] > 0]
negative = data[data["target"] == 0]

print(all.shape, positive.shape, negative.shape, positive.shape[0] + negative.shape[0])

(1831, 22) (176, 22) (1655, 22) 1831


In [6]:
all.drop("target", inplace=True, axis=1)
positive.drop("target", inplace=True, axis=1)
negative.drop("target", inplace=True, axis=1)

print(all.shape, positive.shape, negative.shape, positive.shape[0] + negative.shape[0])

(1831, 21) (176, 21) (1655, 21) 1831


/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


## Positive is the anomalies - Train AutoEncoder with Negatives

In [7]:
INPUT_SIZE = negative.shape[1]
AUTO_ENCODER_SHAPE = 128

In [8]:
train_size = int(negative.shape[0]*90/100)
negative = negative.sample(frac=1)
X_train = negative.iloc[:train_size]
X_test = negative.iloc[-(negative.shape[0] - train_size):]

# force the model to lower the autoencoder reconstruction cost
Y = np.zeros([X_train.shape[0], 1], dtype = float) # ZEROS MATRIX with shape (,1)

print(X_train.shape, X_test.shape, Y.shape, INPUT_SIZE, X_train.shape[0] + X_test.shape[0])

(1489, 21) (166, 21) (1489, 1) 21 1655


## Build the Encoder

In [9]:
input = Input(shape=(INPUT_SIZE,))

input_encoder = layers.Dense(AUTO_ENCODER_SHAPE, activation = 'relu')(input)
hidden2 = layers.Dense(64, activation = 'relu')(input_encoder)
encoded = layers.Dense(32, activation = 'relu')(hidden2)

Instructions for updating:
Colocations handled automatically by placer.


## Build the Decoder

In [10]:
hidden3 = Dense(64, activation='relu')(encoded)
output_decoder = Dense(AUTO_ENCODER_SHAPE, activation='relu')(hidden3)
# decoded = Dense(100, activation='relu')(output_decoder)

In [11]:
from keras import backend as K
from keras.layers import Layer

class ReconstructionCostLayer(Layer):

    def __init__(self,output_dim):
        self.output_dim = output_dim
        super(ReconstructionCostLayer, self).__init__()

    def build(self, input_shape):
        super(ReconstructionCostLayer, self).build(input_shape)  # Be sure to call this at the end

    def call(self, x):
        left = K.slice(x, [0, 0], [-1, AUTO_ENCODER_SHAPE])
        right = K.slice(x, [0, AUTO_ENCODER_SHAPE], [-1, -1])
        return K.sum(K.square(left - right), axis=1)

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim)

### Concatenate Encoder input and Decoder output

In [12]:
input_encoder_plus_output_decoder = layers.concatenate([input_encoder, output_decoder], axis=1)


### Call the custom layer

In [13]:
reconstruction_cost_layer = ReconstructionCostLayer(1)(input_encoder_plus_output_decoder)

# Build the AutoEncoder with Regression Model

In [14]:
model = Model(input, reconstruction_cost_layer)

#### Visualize Model Architecture

In [15]:
adam = optimizers.Adam(lr=0.003, beta_1=0.9, beta_2=0.999, epsilon=1e-4, decay=0.0, amsgrad=True)
model.compile(optimizer=adam, loss='mean_squared_error', metrics=['mse'])


In [16]:
print(model.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 21)           0                                            
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 128)          2816        input_1[0][0]                    
__________________________________________________________________________________________________
dense_2 (Dense)                 (None, 64)           8256        dense_1[0][0]                    
__________________________________________________________________________________________________
dense_3 (Dense)                 (None, 32)           2080        dense_2[0][0]                    
__________________________________________________________________________________________________
dense_4 (D

# Train

In [17]:
tbCallBack = keras.callbacks.TensorBoard(log_dir='./Graph', 
                                         histogram_freq=0, 
                                         write_graph=True, 
                                         write_images=True)

history = model.fit(X_train, Y, epochs=200, batch_size=50, 
                    verbose=1,
                    shuffle=True,
                    callbacks=[tbCallBack],
                   )  

Instructions for updating:
Use tf.cast instead.
Epoch 1/200
1489/1489 [==============================] - 0s 123us/step - loss: 74.5193 - mean_squared_error: 74.5193
Epoch 2/200
1489/1489 [==============================] - 0s 24us/step - loss: 11.8069 - mean_squared_error: 11.8069
Epoch 3/200
1489/1489 [==============================] - 0s 25us/step - loss: 3.9650 - mean_squared_error: 3.9650
Epoch 4/200
1489/1489 [==============================] - 0s 26us/step - loss: 2.0476 - mean_squared_error: 2.0476
Epoch 5/200
1489/1489 [==============================] - 0s 25us/step - loss: 1.6168 - mean_squared_error: 1.6168
Epoch 6/200
1489/1489 [==============================] - 0s 24us/step - loss: 3.0995 - mean_squared_error: 3.0995
Epoch 7/200
1489/1489 [==============================] - 0s 25us/step - loss: 6.7586 - mean_squared_error: 6.7586
Epoch 8/200
1489/1489 [==============================] - 0s 25us/step - loss: 1.6021 - mean_squared_error: 1.6021
Epoch 9/200
1489/1489 [============

1489/1489 [==============================] - 0s 25us/step - loss: 0.0113 - mean_squared_error: 0.0113
Epoch 71/200
1489/1489 [==============================] - 0s 24us/step - loss: 0.0111 - mean_squared_error: 0.0111
Epoch 72/200
1489/1489 [==============================] - 0s 25us/step - loss: 0.0109 - mean_squared_error: 0.0109
Epoch 73/200
1489/1489 [==============================] - 0s 25us/step - loss: 0.0106 - mean_squared_error: 0.0106
Epoch 74/200
1489/1489 [==============================] - 0s 25us/step - loss: 0.0103 - mean_squared_error: 0.0103
Epoch 75/200
1489/1489 [==============================] - 0s 25us/step - loss: 0.0101 - mean_squared_error: 0.0101
Epoch 76/200
1489/1489 [==============================] - 0s 25us/step - loss: 0.0099 - mean_squared_error: 0.0099
Epoch 77/200
1489/1489 [==============================] - 0s 24us/step - loss: 0.0097 - mean_squared_error: 0.0097
Epoch 78/200
1489/1489 [==============================] - 0s 25us/step - loss: 0.0095 - mean_

Epoch 141/200
1489/1489 [==============================] - 0s 25us/step - loss: 0.0037 - mean_squared_error: 0.0037
Epoch 142/200
1489/1489 [==============================] - 0s 24us/step - loss: 0.0036 - mean_squared_error: 0.0036
Epoch 143/200
1489/1489 [==============================] - 0s 24us/step - loss: 0.0036 - mean_squared_error: 0.0036
Epoch 144/200
1489/1489 [==============================] - 0s 23us/step - loss: 0.0036 - mean_squared_error: 0.0036
Epoch 145/200
1489/1489 [==============================] - 0s 24us/step - loss: 0.0035 - mean_squared_error: 0.0035
Epoch 146/200
1489/1489 [==============================] - 0s 24us/step - loss: 0.0035 - mean_squared_error: 0.0035
Epoch 147/200
1489/1489 [==============================] - 0s 24us/step - loss: 0.0035 - mean_squared_error: 0.0035
Epoch 148/200
1489/1489 [==============================] - 0s 24us/step - loss: 0.0034 - mean_squared_error: 0.0034
Epoch 149/200
1489/1489 [==============================] - 0s 25us/step 

# Get Negative (Trained) Reconstruction Costs

In [18]:
THRESHOLD = 0.4

def report_cost(result):
    tot = len(result)
    greater = len(np.where(result > THRESHOLD)[0])
    print("Count: {}\nGreater Than Threeshold: {} {}%\nMean: {}\nStd: {}\nMin: {}\nMax: {}"\
          .format(tot,
                  greater,
                  (tot - greater) * 100 / tot,
                  result.mean(),
                  result.std(),
                  result.min(),
                  result.max()))

In [19]:
reconstruction_costs = model.predict(X_train)
report_cost(reconstruction_costs)

Count: 1489
Greater Than Threeshold: 0 100.0%
Mean: 0.028791146352887154
Std: 0.03308597579598427
Min: 0.00029613656806759536
Max: 0.22811323404312134


In [20]:
# unique, counts = np.unique(reconstruction_costs, return_counts=True)
# dict(zip(unique, counts))

# Get Negative (New) Reconstruction Costs

In [21]:
reconstruction_costs = model.predict(X_test)
report_cost(reconstruction_costs)

Count: 166
Greater Than Threeshold: 3 98.19277108433735%
Mean: 0.043408434838056564
Std: 0.10676226019859314
Min: 0.0005306086968630552
Max: 0.9533023834228516


In [22]:
# unique, counts = np.unique(reconstruction_costs, return_counts=True)
# dict(zip(unique, counts))

# Get Positive Reconstruction Costs

In [23]:
reconstruction_costs = model.predict(positive)
report_cost(reconstruction_costs)

Count: 176
Greater Than Threeshold: 84 52.27272727272727%
Mean: 1.5404709577560425
Std: 3.19538950920105
Min: 0.021078500896692276
Max: 22.383573532104492


In [24]:
# unique, counts = np.unique(reconstruction_costs, return_counts=True)
# dict(zip(unique, counts))